In [1]:
d=15

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from nselib import capital_market 
import pandas as pd
import matplotlib.pyplot as plt 
import os
symbol = "ADANIENT"

fdate_for_trgt_stp=f'01-01-20{d}'
tdate_for_trgt_stp=f'01-01-20{d+2}'
fdate_for_profit=f'01-01-20{d+2}'
tdate_for_profit=f'01-01-20{d+3}'
stp = np.arange(0,10,0.01)
trgt =np.arange(0,50,0.1)


In [3]:
def replace_commas(x):
    if isinstance(x, str):
        return x.replace(',', '')
    return x

In [4]:

def getdata(symbol,fdate,tdate):
    data = capital_market.price_volume_and_deliverable_position_data(symbol=symbol, from_date=fdate, to_date=tdate)
    df = pd.DataFrame(data)
    df1 = df[["Date", "Symbol", "OpenPrice", "HighPrice", "LowPrice", "ClosePrice"]]
    df1=df1.drop_duplicates(['Date'])


    # Apply the function to columns
    df1["HighPrice"] = df1['HighPrice'].apply(replace_commas)
    try:
        df1["OpenPrice"] = df1['OpenPrice'].apply(replace_commas)
        df1["ClosePrice"] = df1['ClosePrice'].apply(replace_commas)
        df1["LowPrice"] = df1['LowPrice'].apply(replace_commas)
    except:
        print("Replace done!")

    # Convert columns to numeric
    df1["HighPrice"] = pd.to_numeric(df1["HighPrice"], errors='coerce')
    df1["OpenPrice"] = pd.to_numeric(df1["OpenPrice"], errors='coerce')
    df1["ClosePrice"] = pd.to_numeric(df1["ClosePrice"], errors='coerce')
    df1["LowPrice"] = pd.to_numeric(df1["LowPrice"], errors='coerce')
    df1['Date']=pd.to_datetime(df1['Date'])
    df1["day"] = df1['Date'].dt.day
    df1["month"] = df1['Date'].dt.month
    df1["year"] = df1['Date'].dt.year
    df1=df1.groupby(["year","month"])
    month=[]
    year=[]
    OpenPrice=[]
    ClosePrice=[]
    HighPrice=[]
    LowPrice=[]
    symbole=[]

    for name , group in df1:   
        symbole.append(group['Symbol'].values[0])
        month.append(name[1])
        year.append(name[0])
        OpenPrice.append(group['OpenPrice'].values[0])
        ClosePrice.append(group['ClosePrice'].values[-1:][0])
        HighPrice.append(group['HighPrice'].max())
        LowPrice.append(group['LowPrice'].min())
    df2={"symbol":symbole,"month":month,"year":year,"OpenPrice":OpenPrice,"HighPrice":HighPrice,"LowPrice":LowPrice,"ClosePrice":ClosePrice}
    df2=pd.DataFrame(df2)
    df1=df2 


    # Calculate percentage changes
    df1["High%"] = (df1["HighPrice"] - df1["OpenPrice"]) * 100 / df1["OpenPrice"]
    df1["High%"] = df1["High%"].apply(lambda x: "{:.3f}".format(x))

    df1["Low%"] = (df1["LowPrice"] - df1["OpenPrice"]) * 100 / df1["OpenPrice"]
    df1["Low%"] = df1["Low%"].apply(lambda x: "{:.3f}".format(x))

    df1["Close%"] = (df1["ClosePrice"] - df1["OpenPrice"]) * 100 / df1["OpenPrice"]
    df1["Close%"] = df1["Close%"].apply(lambda x: "{:.3f}".format(x))

    # Convert percentage columns back to numeric
    df1["Low%"] = pd.to_numeric(df1["Low%"], errors='coerce')
    df1["High%"] = pd.to_numeric(df1["High%"], errors='coerce')
    df1["Close%"] = pd.to_numeric(df1["Close%"], errors='coerce')

    # Calculate target and stoploss percentages
    df1.loc[abs(df1['High%']) <= abs(df1['Low%']), 'target_wrt_h&l%'] = abs(df1['Low%'])
    df1.loc[abs(df1['High%']) > abs(df1['Low%']), 'target_wrt_h&l%'] = abs(df1['High%'])
    df1.loc[abs(df1['High%']) >= abs(df1['Low%']), 'stoploss_wrt_h&l%'] = abs(df1['Low%'])
    df1.loc[abs(df1['High%']) < abs(df1['Low%']), 'stoploss_wrt_h&l%'] = abs(df1['High%'])

    # Determine profit decider
    df1.loc[abs(df1['target_wrt_h&l%']) == abs(df1['High%']), 'profit_decider'] = abs(df1['Low%'])
    df1.loc[abs(df1['target_wrt_h&l%']) == abs(df1['Low%']), 'profit_decider'] = abs(df1['High%'])
    
    try:
        df1.to_csv(f"excel/in_details/{symbol}/{fdate}_{tdate}.csv",index=False)
    except:
        os.mkdir(f"excel/in_details/{symbol}")
        df1.to_csv(f"excel/in_details/{symbol}/{fdate}_{tdate}.csv",index=False)










In [5]:
getdata(symbol,fdate_for_trgt_stp,tdate_for_trgt_stp)
getdata(symbol,fdate_for_profit,tdate_for_profit)

C:\Users\vivekant kumar\AppData\Local\Temp\ipykernel_7096\109704895.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["HighPrice"] = df1['HighPrice'].apply(replace_commas)
C:\Users\vivekant kumar\AppData\Local\Temp\ipykernel_7096\109704895.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["OpenPrice"] = df1['OpenPrice'].apply(replace_commas)
C:\Users\vivekant kumar\AppData\Local\Temp\ipykernel_7096\109704895.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [6]:
def prfdata(symbol,stploss,trgt):
    df1=pd.read_csv(f"excel/in_details/{symbol}/{fdate_for_trgt_stp}_{tdate_for_trgt_stp}.csv")
   
    df1['stploss']=df1['target_wrt_h&l%'].apply(lambda x:stploss)
    df1['trgt']=df1['target_wrt_h&l%'].apply(lambda x:trgt)
    df1['stploss']=pd.to_numeric(df1['stploss'])
    df1['trgt']=pd.to_numeric(df1['trgt'])
    df1.loc[((abs(df1['target_wrt_h&l%']) >= df1['trgt']) & ((abs(df1['profit_decider'])) < df1['stploss'])), 'profit'] = (df1['trgt']-df1['stploss'])
    df1.loc[(abs(df1['profit_decider']) >= df1['stploss']) , 'profit'] = -(2 * df1['stploss'])
    df1.loc[((abs(df1['target_wrt_h&l%']) < abs(df1['trgt'])) & ((abs(df1['profit_decider'])) < df1['stploss']) ) , 'profit'] = (abs(df1['Close%'])-df1['stploss'])
    df1['profit'] = pd.to_numeric(df1['profit'])

    profit = df1['profit'].sum()
   

    data = 1 + df1['profit']/100
    
    overallprofit =  data.cumprod()[-1:].values[0]
    formatted_overallprofit = "{:.4f}".format(overallprofit)
    

    

    
    dic={"tprofit":profit,"cprofit":formatted_overallprofit}
    
    
    return dic


In [7]:
def prfdata1(symbol,stploss,trgt):
    df1=pd.read_csv(f"excel/in_details/{symbol}/{fdate_for_profit}_{tdate_for_profit}.csv")
   
    df1['stploss']=df1['target_wrt_h&l%'].apply(lambda x:stploss)
    df1['trgt']=df1['target_wrt_h&l%'].apply(lambda x:trgt)
    df1['stploss']=pd.to_numeric(df1['stploss'])
    df1['trgt']=pd.to_numeric(df1['trgt'])
    df1.loc[((abs(df1['target_wrt_h&l%']) >= df1['trgt']) & ((abs(df1['profit_decider'])) < df1['stploss'])), 'profit'] = (df1['trgt']-df1['stploss'])
    df1.loc[(abs(df1['profit_decider']) >= df1['stploss']) , 'profit'] = -(2 * df1['stploss'])
    df1.loc[((abs(df1['target_wrt_h&l%']) < abs(df1['trgt'])) & ((abs(df1['profit_decider'])) < df1['stploss']) ) , 'profit'] = (abs(df1['Close%'])-df1['stploss'])
    df1['profit'] = pd.to_numeric(df1['profit'])
    df1['cumsum']=df1['profit'].cumsum()
    df1['cumprod']=(df1['profit']/100+1).cumprod()

    profit = df1['profit'].sum()
    # print(f"Total profit = {profit}% in 2 years")
   
       
    dic={"tprofit":profit,"cprofit":df1['cumprod'][-1:]}
    df1.to_csv(f"excel/in_details/{symbol}/{fdate_for_profit}_{tdate_for_profit}.csv",index=False)    
    return dic


In [8]:


def fstp_ftrgt():
    max=0
    fstp=0
    ftrgt=0

    for estp in stp:
        print(estp)
        for etrgt in trgt:
            cc=prfdata(symbol,estp,etrgt)
            
            
            if(max<cc['tprofit']):            
                max=cc['tprofit']
                fstp=estp
                ftrgt=etrgt
    dic={"fstp":fstp,"ftrgt":ftrgt}
    return dic 
 
                
                

        
            


In [9]:
dic=fstp_ftrgt()
fstp=dic['fstp']
ftrgt=dic['ftrgt']




0.0


IndexError: index 0 is out of bounds for axis 0 with size 0

In [10]:
prf_dic=prfdata1(symbol,fstp,ftrgt)
print(f"total profit={prf_dic['tprofit']}")
print(f"cumprod profit={prf_dic['cprofit'].values[0]}")

total profit=14.095999999999993
cumprod profit=1.0652371228109077


In [11]:
vivedhara = {
    "symbol": [symbol],
    "tprofit": [prf_dic['tprofit']],
    "cprofit": [prf_dic['cprofit'].values[0]],
    "fstp": [fstp],
    "ftrgt": [ftrgt],
    "fdate": [fdate_for_profit],
    "tdate": [tdate_for_profit],
    "fdate_trgt_stp": [fdate_for_trgt_stp],
    "tdate_trgt_stp": [tdate_for_trgt_stp]
}

try:
    final_data = pd.DataFrame(vivedhara)
    # final_data.to_csv(f"excel/all_in_one{symbol}.csv",index=False)
    pre_data=pd.read_csv(f"excel/all_in_one/{symbol}.csv")
    pre_data=pd.concat([pre_data,final_data])
    pre_data.to_csv(f"excel/all_in_one/{symbol}.csv",index=False)
except  :
 final_data = pd.DataFrame(vivedhara)
 final_data.to_csv(f"excel/all_in_one/{symbol}.csv",index=False)

